In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from transformers import pipeline
import numpy as np

In [2]:
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [3]:
key_word = "iran" 

# Insert key word to scrape for

In [4]:
websites = ['https://www.bbc.co.uk/', "https://www.theguardian.com/"]

In [5]:
checked_articles = set()
mentions = []
headlines = []
links = []
summaries = []

session = requests.Session()

for website in websites:

    s = session.get(website)
    s.encoding = "utf-8"
    site_soup = BeautifulSoup(s.text, "html.parser")

    raw_stories = site_soup.select("h3:has(a)") or site_soup.select("a:has(h3)")

    for story in raw_stories:
        
        if not story.find("a"):
            article_link = website + (story.get("href"))[1:]
        
        else:
            article_link = website + (story.find("a").get("href"))[1:]
            
        if article_link in checked_articles:
            continue
            
        else:
            checked_articles.add(article_link)
            
        article_ses = session.get(article_link)
        article_soup = BeautifulSoup(article_ses.text, "html.parser")

        headline = story.text
        article = article_soup.find("article")
        
        if not article:
            continue
        
        headline = story.text
        article_text = article.text
        
        if key_word.lower() in article_text.lower():
            mentions.append(article_text.lower().count(key_word.lower()))
            headlines.append(headline)
            links.append(article_link)

            article_summary = summarizer(article_text[:1024], max_length = 130, min_length = 30, do_sample = False)
            summaries.append(article_summary[0]["summary_text"])
            
session.close()

In [6]:
relevant_articles_df = pd.DataFrame(
    {"Headline": headlines,
     "Article Link":links,
     "No. Mentions":mentions,
     "Summary":summaries})

relevant_articles_df

,Headline,Article Link,No. Mentions,Summary
0,Live. BBC Verify Live: What videos tell us abo...,https://www.bbc.co.uk/news/live/c8xvrgz5vvkt,4,Iranian media publish CCTV recording of Tehra...
1,Katy Perry and Orlando Bloom confirm split,https://www.bbc.co.uk/news/articles/cj61x8r552ko,1,Katy Perry and Orlando Bloom have been romant...
2,Kuenssberg: Starmer's first year ends in shamb...,https://www.bbc.co.uk/news/articles/cp8m9vdz2m4o,3,Kuenssberg: Starmer's first year ends in sham...
3,IsraelIsrael steps up deadly bombardment of Ga...,https://www.theguardian.com/world/2025/jul/03/...,2,Medics say about 90 people killed since Wedne...
4,The Starmtrooper rebellionWelfare bill vote sh...,https://www.theguardian.com/politics/2025/jul/...,1,"The government won the vote 336-261, but only..."
